<a href="https://colab.research.google.com/github/botchagalupe/Digital_Deming/blob/main/Prototype_Digital_Deming_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up your local environment 

On intial run of "import colab_env" it will create vars.env in the root of your Google Drive

COLAB_ENV = Active

export OPENAI_API_KEY = "your_key"

export SERPAPI_API_KEY = "your_key"


#Backend Code

Install my local packages

In [ ]:
pip install OpenAI langchain pydantic faiss-gpu colab-env unstructured chromadb tiktoken pdf2image faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.5 

Yo, let's import our local env so we don't have to show the world our keys

In [ ]:
import colab_env

Mounted at /content/gdrive


Let's import our python modules

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import magic
import os
import nltk
import textwrap



**Define Functions**

In [ ]:
def print_it():

  global content
  global sources

  #print("content ", content)
  #print("sources ",sources)

In [ ]:
def format_book(format_str):

  global content
  
  content += format_str
  
  print_it()

In [ ]:
def call_query(query):
  result = qa({"query": query})
  #print(result)

  # Result is type dict, content is type str, sources is type list 

  global content
  global sources

  content += result['result']
  sources += result['source_documents']

  print_it()

Let's set the temperature of our AI models

In [ ]:
llm = OpenAI(temperature=0)
chat = ChatOpenAI(temperature=0)

Let's specify the directory and file type for our Demming source docs

In [ ]:
loader = DirectoryLoader('/content/gdrive/MyDrive/Projects/Digital_Deming/src_content', glob='**/*.pdf')

Load our documents


In [ ]:
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Check how many documents are loaded

In [ ]:
len (documents)

1

Let's split our document up into chunks

In [ ]:
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    separators=["\n\n", "\n", " ", ""]
)

Let' create a variable named texts, and split these documents into it

In [ ]:
texts = text_splitter.split_documents(documents)

Let's tell it that we want it to use OpenAI embeddings, and tell it what env variable the key is held

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

Let's create a variable named docsearch, and pull from Chroma vectorstore including texts, and the embeddings)

In [ ]:
docsearch = Chroma.from_documents(texts, embeddings)

Let's define qa and pull it from our local vectorDB

In [ ]:
#qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=False)
#qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Provide responses in Markdown using # to indicate a heading and ## to indicate a subheading#

{context}

Question: {question}
Answer as if you are a quality management consultant. Use specific dates and references. Use bullet points and tabels where appropriate. Use markdown to produce the output.Reference iource documents:

"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs, return_source_documents=True)

define qa chain, which pulls from VectorDBQA, and use the LLM chain named OpenAI. We specify that it "Stuff"'s all the data in. We could use "map_reduce" instead to optimize for performance and scale. Let's tell it to use the vectorstore we defined earlier named docsearch

note, i'm trying RetrievalQA because VectorDBQA is depricated

Inital Call for Book Introduction

In [ ]:
query = "write a breif overview about a book about Kaoru Ishikawa, JUSE, Deming, and TQM)"
result = qa({"query": query})
content = result['result']
sources = result['source_documents']

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
* Definition of Total Quality Management (TQM)
* Brief introduction of the significance of TQM in the modern business world
* Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
* Thesis Statement

In [ ]:
query = "write a breif introduction to TQM)"
call_query(query)

In [ ]:
query = "write a definition of Total Quality Management (TQM)"
call_query(query)

In [ ]:
query = "write a paragraph about the significance of TQM in the modern business world"
call_query(query)

In [ ]:
query = "write a paragraph about Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
call_query(query)

In [ ]:
query = "write a Thesis statement about Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
call_query(query)

## II. Early History of Quality Management
- Brief description of quality management before TQM
- Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance
- The rise of statistical process control and its significance

In [ ]:
query = "write about the early History of Quality Management"
call_query(query)

In [ ]:
query = "write a brief description of quality management before TQM"
call_query(query)

In [ ]:
query = "write Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance"
call_query(query)

In [ ]:
query = "write about the rise of statistical process control and its significance"
call_query(query)

## III. The Birth and Evolution of TQM
- Introduction to the concept of TQM
- Overview of the principles and practices of TQM
- The role of TQM in post-World War II Japan's industrial comeback
- Adoption and evolution of TQM in the United States and around the world

In [ ]:
query = "write a breif introduction about the Birth and Evolution of TQM"
call_query(query)

In [ ]:
query = "write an Introduction to the concept of TQMM"
call_query(query)

In [ ]:
query = "write an Overview of the principles and practices of TQM"
call_query(query)

In [ ]:
query = "write about The role of TQM in post-World War II Japan's industrial comeback"
call_query(query)

In [ ]:
query = "write about the Adoption and evolution of TQM in the United States and around the world"
call_query(query)

## IV. The Role of Kaoru Ishikawa in the Development of TQM
- Introduction to Ishikawa: his background and career
- Ishikawa's contribution to the development of Quality Circles
- The development and importance of the Ishikawa (cause-and-effect) Diagram
- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'


In [ ]:
query = "write about The Role of Kaoru Ishikawa in the Development of TQM"
call_query(query)

In [ ]:
query = "write about an introduction to Ishikawa: his background and career"
call_query(query)

In [ ]:
query = "write about Ishikawa's contribution to the development of Quality Circles"
call_query(query)

In [ ]:
query = "write about the development and importance of the Ishikawa (cause-and-effect) Diagram"
call_query(query)

In [ ]:
query = "write about Ishikawa's philosophical contribution to TQM: the 'next process is your customer'"
call_query(query)

## V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
- Background of JUSE and its role in the development of quality management in Japan
- Introduction of the Deming Prize by JUSE and its significance
- JUSE's role in promoting TQM and its principles in Japan and globally


In [ ]:
query = "write a breif introduction about the Role of the Japanese Union of Scientists and Engineers (JUSE)"
call_query(query)

In [ ]:
query = "write about the background of JUSE and its role in the development of quality management in Japan"
call_query(query)

In [ ]:
query = "write about the introduction of the Deming Prize by JUSE and its significance"
call_query(query)

In [ ]:
query = "write about JUSE's role in promoting TQM and its principles in Japan and globally"
call_query(query)

## VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
- Analysis of the role TQM plays in today's business and management practices
- The legacy of Ishikawa's methods and tools in current quality management
- Influence of JUSE's principles in contemporary quality initiatives and awards


In [ ]:
query = "write a breif intro to the Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practicess"
call_query(query)

In [ ]:
query = "write about the legacy of Ishikawa's methods and tools in current quality management"
call_query(query)

In [ ]:
query = "write an analysis of the role TQM plays in today's business and management practices"
call_query(query)

In [ ]:
query = "write about the Influence of JUSE's principles in contemporary quality initiatives and awards"
call_query(query)

## VII. Conclusion
- Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
- Restatement of Thesis
- Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape

In [ ]:
query = "write about: Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management"
call_query(query)

In [ ]:
query = "Restatement of Thesis - Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
call_query(query)

FAQ

In [ ]:
query = "who won the deming prize"
call_query(query)

In [ ]:
query = "when was the QC Circle Seminar for Top Management helde"
call_query(query)

In [ ]:
content = "" 
format_book("# Hellow Word<br>")

In [ ]:
print("content ", content)
print("sources ",sources)

content  #Hellow Word<br>
sources  [Document(page_content='Deming Application Prize. It was the first construction company to win the prize. (Nov.)\n\n\uf09f A committee to research QC in the\n\nservice industry was established. (Nov.,\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan\n\nYear\n\nProf. Ishikawa’s Life\n\nMajor Events in the Development of QC\n\ncommissioned by the Ministry of the Environment.)\n\n\uf09f His book Control Points of Quality (joint authorship) (Zeimukeiri Kyokai) was awarded Nikkei Quality Control Literature Prize.\n\nJUSE)\n\n\uf09f The 28th QCS: New product planning\n\nand market research.\n\n\uf09f The 29th QCS: Future Quality\n\nControl—challenges in the 1980s.\n\n1980\n\n\uf09f He became a council member of the Japan Standards Association. (–84)\n\n\uf09f He became a council member of the Japanese Society for Quality Control. (–82)\n\n\uf09f He became the chair of the Nuclear\n\nPower Plant Quality Assurance Review Committee. (MI